In [89]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

from geopy import distance


## Data from zenodo archive

https://zenodo.org/record/7923702

In [10]:
january=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190101_20190131.csv')
january.head()

C:\Users\a.salgas.ISAE-SUPAERO\AppData\Local\Temp\ipykernel_7144\2227663821.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  january=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190101_20190131.csv')


,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,HVN19,NaN,888152,NaN,NaN,YMML,LFPG,2018-12-31 00:43:16+00:00,2019-01-01 04:56:29+00:00,2019-01-01 00:00:00+00:00,-37.659485,144.804421,304.8,48.995316,2.610802,-53.34
1,CCA839,NaN,780ad1,NaN,NaN,YMML,LEBL,2018-12-31 00:53:08+00:00,2019-01-01 06:01:42+00:00,2019-01-01 00:00:00+00:00,-37.692123,144.841997,304.8,41.036124,2.063557,1112.52
2,CES219,NaN,780b7e,B-5936,A332,YSSY,EDDF,2018-12-31 01:05:29+00:00,2019-01-01 04:09:29+00:00,2019-01-01 00:00:00+00:00,-33.930908,151.171987,0.0,50.045563,8.588923,-15.24
3,AEA040,NaN,34444e,EC-LVL,A332,LEMD,LEMD,2018-12-31 01:07:21+00:00,2019-01-01 03:32:59+00:00,2019-01-01 00:00:00+00:00,40.534756,-3.575426,609.6,40.475728,-3.538347,411.48
4,CXA825,NaN,780d75,B-2760,B788,YSSY,LFPG,2018-12-31 01:18:29+00:00,2019-01-01 04:32:28+00:00,2019-01-01 00:00:00+00:00,-33.954254,151.178041,0.0,48.996091,2.625805,22.86


In [11]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/Utilities")

import open_sky



In [12]:
os_flights_2019=open_sky.FlightOpenSky()
os_flights_2019.load_new_file('D:/a.salgas/Documents/opensky_2019/flightlist_20190101_20190131.csv',0.01) # ==> the file was extracted from opensky zenodo archive with a notebook that can be found in Utilities folder
os_flights_2019.preprocess_data()


D:\AeroSCOPE\aeroscope/Utilities\flights.py:31: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(relative_path).sample(frac=sample_ratio, random_state=6)


48.63668142624097% of flights deleted after removing flights with no origin and destination. 11020 Flights in the dataset


D:\AeroSCOPE\aeroscope/Utilities\open_sky.py:51: DtypeWarning: Columns (4,5,17,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  ac_ref = pd.read_csv(data_path, sep=',')


Size of df after_grouping: 9500; number of flights: 11020
Size of df after ac info: 9500; number of flights 11020
Size of df after arpt info: 9459, number of flights: 10971


In [81]:
os_df=os_flights_2019.df

os_df

,airline_iata,origin,dest,aircraft_type,n_flights,acft_icao,acft_class,seymour_proxy,origin_iata,origin_lon,origin_lat,origin_country,origin_continent,dest_iata,dest_lon,dest_lat,dest_country,dest_continent,distance_km
0,AAF,EBBR,LPPT,A320,1,A320,NB,A320,BRU,4.484440,50.901402,BE,EU,LIS,-9.135920,38.781300,PT,EU,1718.675355
1,AAF,EHAM,LPPT,A320,1,A320,NB,A320,AMS,4.763890,52.308601,NL,EU,LIS,-9.135920,38.781300,PT,EU,1847.468375
2,AAF,LFPO,DAAG,A320,1,A320,NB,A320,ORY,2.379440,48.723330,FR,EU,ALG,3.214531,36.693886,DZ,AF,1338.047133
3,AAF,LFPO,LIMC,A320,1,A320,NB,A320,ORY,2.379440,48.723330,FR,EU,MXP,8.728110,45.630600,IT,EU,591.142052
4,AAF,LPPT,LIPZ,A320,1,A320,NB,A320,LIS,-9.135920,38.781300,PT,EU,VCE,12.351900,45.505299,IT,EU,1919.300938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9493,ZZZ,YTYA,YMMB,C172S,1,C172,PP,C172,NaN,145.179406,-38.266826,AU,OC,MBW,145.102005,-37.975800,AU,OC,33.008873
9494,ZZZ,YWIO,YWIO,C208,1,C208,TP,C208,NaN,150.670760,-34.219990,AU,OC,NaN,150.670760,-34.219990,AU,OC,0.000000
9497,ZZZ,YWSG,YZCE,C172,1,C172,PP,C172,NaN,152.460007,-27.098301,AU,OC,NaN,152.378454,-27.429915,AU,OC,37.622296
9498,ZZZ,YYEA,YMEN,A139,1,A139,HE,zzz,NaN,145.323056,-37.175904,AU,OC,MEB,144.901993,-37.728100,AU,OC,71.721691


In [30]:
fleet=pd.read_excel('data/planespotters_fleet.xlsx')
fleet['Seat Total']=fleet['Seat Total'].replace(0, np.nan)
fleet.dropna(subset=['Seat Total'], inplace=True)

# Adding the name/icao code match table
match=pd.read_csv('data/fleet_match.csv', sep=';')
fleet=fleet.merge(match, left_on='Aircraft Type', right_on='spotter_name', how='right').drop(columns='spotter_name')

aircraft_data=fleet.groupby(['Aircraft Type','ICAO_AC'])['Seat Total'].mean().reset_index().drop(columns='Aircraft Type')

In [69]:
aircraft_data.head()

,ICAO_AC,Seat Total
0,AT43,46.897119
1,AT45,46.897119
2,AT72,70.137580
3,AT75,70.137580
4,AT76,70.137580


In [82]:
os_df=os_df.merge(aircraft_data, left_on='acft_icao', right_on='ICAO_AC', how='left')
os_df['Seat Total']=os_df['n_flights']*os_df['Seat Total']
os_df=os_df.groupby(['origin_iata','origin_lon','origin_lat','origin_country','origin_continent','dest_iata','dest_lon','dest_lat','dest_country','dest_continent','airline_iata','acft_icao','acft_class','seymour_proxy'])[['Seat Total','n_flights']].sum().reset_index().rename(columns={'Seat Total':'seats'})

In [83]:
column_mapping = {
    'origin_iata': 'iata_departure',
    'dest_iata': 'iata_arrival',
    'origin_lon': 'departure_lon',
    'origin_lat': 'departure_lat',
    'origin_country': 'departure_country',
    'origin_continent': 'departure_continent',
    'dest_lon': 'arrival_lon',
    'dest_lat': 'arrival_lat',
    'dest_country': 'arrival_country',
    'dest_continent': 'arrival_continent'
}                          


os_df.rename(columns=column_mapping, inplace=True)
os_df['source']='OpenSky'

In [84]:
iata_icao_convert=pd.read_csv('data/iata_icao_airline.csv', sep=';')
iata_icao_convert=iata_icao_convert.sort_values(by=['IATA'], ascending=False).drop_duplicates(subset='ICAO', keep='first')

In [85]:
os_df=os_df.merge(iata_icao_convert[['IATA','ICAO']], left_on='airline_iata', right_on='ICAO', how='left')

In [86]:
os_df['IATA']=os_df['IATA'].fillna(os_df['airline_iata'])
os_df=os_df.drop(columns=['airline_iata','ICAO']).rename(columns={'IATA':'airline_iata'})

In [90]:
os_df.loc[:, "distance_km"] = os_df.progress_apply(
            lambda x: distance.distance((float(x.departure_lat), float(x.departure_lon)),
                                        (float(x.arrival_lat), float(x.arrival_lon))).km if not (pd.isna(x.departure_lat) or pd.isna(x.departure_lon)
                                                                                           or pd.isna(x.arrival_lon) or pd.isna(x.arrival_lat)) else 0, axis=1)


  0%|          | 0/6916 [00:00<?, ?it/s]

In [91]:
os_df
os_df['ask'] = os_df["distance_km"]*os_df["seats"]
os_df['rpk'] = os_df['ask']*0.824

Load factor source: https://www.icao.int/annual-report-2019/Pages/the-world-of-air-transport-in-2019.aspx

In [93]:
os_df['seats'].sum()

1103876.31452475